In [ ]:
!pip install -U pycld2

In [34]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
import pycld2 as cld2

sys.path.append('../')

In [35]:
import configparser
configParser = configparser.RawConfigParser()   
configFilePath = r'config.txt'
configParser.read(configFilePath)
csvpath = configParser.get('paths', 'csvpath')
outpath = configParser.get('paths', 'outpath')
outpath,csvpath

('./processed/', './data/')

In [37]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [4]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [5]:
def find_eng(string): 
    text = re.findall('[a-zA-Z0-9]',string)
    return "".join(text) # converting return value from list to string

In [6]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [7]:
def remove_eng(data):
    return ' '.join(re.sub(u"[^ሀ-፼]", " ", data).split()).strip()
     

In [8]:
def remove_English_letters(content):
    for idx,cnt in enumerate(content):
        content[idx] = re.sub(r'\s*[A-Za-z]+\b','', cnt)
    return content

In [9]:
def remove_emoji(news):
    news = emoji.demojize(news)
    re.sub('(:\S+)(@\w+)','', news)
    return news

In [10]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [38]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

# Read each CSV files and write it to text file

In [ ]:
remove = ['\xa087','\xa0', '|', '/','…','«','"','“','”','»','-',
          '$','%','^','&','*','›','’','‘','‹','<','>','`','´','~','=','+']
# remove output file if it exists, then append each csv to a single file output
outfile = outpath+"all_csv_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
# all csv files, to controll progress par
allnews = []
for news in glob.glob(csvpath+"/*.csv"):
    allnews.append(news)
    
# read each files and write to a file system 

with open (outfile,"a", encoding="utf-8") as all_sentences:
    
    lines_seen = set() # holds lines already seen
    for news in tqdm.tqdm(allnews,  position=0, leave=True):
        data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
        data = data[data.Content.duplicated()==False].reset_index()
        #data.Content = data.Content.apply(lambda x: remove_url(x))
        #data.Content = data.Content.apply(lambda x: remove_emoji(x))
        data.Content = data.Content.apply(lambda x: remove_eng(x))
        for token in remove:
            data.Content = data.Content.apply(lambda x: x.replace(token,' '))  
        for text in data.Content:
            for s in amseg.tokenize_sentence(text):
                isReliable, textBytesFound, details  = cld2.detect(s)
                if details[0][1] =='am' and s not in lines_seen: # check if line is not duplicate:
                    all_sentences.write(' '.join(t.text for t in amseg.amharic_tokenizer(s))+'\n')
                    lines_seen.add(s)
                           



# downloading online text file to 'data' folder with new name

In [ ]:
#download text file from online
!wget -O data/amharic1.txt "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/am.txt"
!wget -O data/amharic2.txt  "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_E-Bible/amharic.txt"
!wget -O data/amharic3.txt "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_Ethiopic_Bible/amharic.txt"
!wget -O data/amharic4.txt "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_History/amharic.txt"
!wget -O data/amharic5.txt "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_JW_Bible/amharic.txt"
!wget -O data/amharic6.txt "https://raw.githubusercontent.com/adtsegaye/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_JW_Daily_Quote/amharic.txt"


In [ ]:
#downloading zipped files to data filder with renamed file name and in gz format
!wget -O data/am1.txt.zip "https://object.pouta.csc.fi/OPUS-Tanzil/v1/mono/am.txt.gz" 
!wget -O data/am2.txt.zip "https://object.pouta.csc.fi/OPUS-JW300/v1b/mono/am.txt.gz"
!wget -O data/am3.txt.zip "https://object.pouta.csc.fi/OPUS-GNOME/v1/mono/am.txt.gz"
!wget -O data/am4.txt.zip "https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/am.tok.gz"
!wget -O data/am5.txt.zip "https://object.pouta.csc.fi/OPUS-Tanzil/v1/mono/am.txt.gz"
!wget -O data/am6.txt.zip "https://object.pouta.csc.fi/OPUS-Mozilla-I10n/v1/mono/am.tok.gz"
!wget -O data/am7.txt.zip "https://object.pouta.csc.fi/OPUS-TED2020/v1/mono/am.txt.gz"
!wget -O data/am8.txt.zip "https://object.pouta.csc.fi/OPUS-GlobalVoices/v2018q4/mono/am.txt.gz"


In [32]:
#from zipfile import ZipFile
#with ZipFile('data/am8.txt.zip', 'r') as my_zip:
#    my_zip.extractall('data')

# unzip all zipped files on data folder

In [30]:
#changing working directory to 'data' folder that contain all zipped files
import os
import gzip
import shutil
#search_path = os.getcwd()
search_path = os.chdir('data')#change work directory

file_type = ".gz"
for fname in os.listdir(path =search_path):
    if fname.endswith(file_type):
        with gzip.open(fname,'rb') as f_in:
            with open(fname+'.txt','wb') as f_out:
                shutil.copyfileobj(f_in,f_out)

In [31]:
#Return to back=>normalization directory
os.chdir('..')


# merging online text files as 'all_online_sentences' & save to outpath

In [ ]:

outfile = csvpath+"all_online_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
with open (outfile,"a", encoding="utf-8") as result:
    
    for file in glob.glob(outpath+"/*.txt"):
        with open(file,encoding="utf-8") as infile:
            for line in infile:
                result.write(line)


# preprocess merged online text file

In [ ]:
outfile = outpath+"all_online_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None

with open (outfile,"a", encoding="utf-8") as all_online_sentences:
    lines_seen = set() # holds lines already seen
    data = pd.read_csv(csvpath+"all_online_sentences.txt",names=["Content"] ,encoding="utf-8")#file that we get from colab
    data = data[data.Content.duplicated()==False].reset_index()
    data.Content = data.Content.apply(lambda x: remove_eng(x))
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token,' '))  
    for text in data.Content:
        for s in amseg.tokenize_sentence(text):
            isReliable, textBytesFound, details  = cld2.detect(s)
            if details[0][1] =='am' and s not in lines_seen: # check if line is not duplicate:
                all_online_sentences.write(' '.join(t.text for t in amseg.amharic_tokenizer(s))+'\n')
                lines_seen.add(s)
    

# merging preprocessed csv text file and online text file to 'all_in_one'

In [ ]:
outfile = outpath+"all_in_one.txt"
os.remove(outfile) if os.path.exists(outfile) else None
with open (outfile,"a", encoding="utf-8") as result:
    
    for file in glob.glob(outpath+"/*.txt"):
        with open(file,encoding="utf-8") as infile:
            for line in infile:
                result.write(line)


# devide text file to train and test 

In [ ]:
with open(outpath+"all_in_one.txt",'r',encoding="utf-8")as line:
    no_of_line = line.read().splitlines()
len(no_of_line)

In [ ]:
import random
with open(outpath+"all_in_one.txt", "r",encoding="utf-8") as f:    
    data = f.read().split('\n')
    random.shuffle(data)
#Assume we have 117552 line of text and 90:10 ratio for  train and test respectivily
    test_data = data[:11755]
    train_data = data[11755:]

In [ ]:
len(train_data), len(test_data)

In [ ]:
#data = ["snet1", "sent2", "sent3","sent4","sent5","sent6","sent7","sent8","sent9","sent10"]
#random.shuffle(data)
#print(data[:len(data)*0.9])